# GAN Trainer

Takes the gan model and trains it to produce optimal weights to be loaded in

## Setup the dataset


In [1]:
!git clone https://github.com/ed22699/super-resolution-model-tests.git

Cloning into 'super-resolution-model-tests'...
remote: Enumerating objects: 23, done.
remote: Counting objects: 100% (23/23), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 23 (delta 7), reused 18 (delta 5), pack-reused 0 (from 0)
Receiving objects: 100% (23/23), 11.89 KiB | 11.89 MiB/s, done.
Resolving deltas: 100% (7/7), done.


In [2]:
!cd super-resolution-model-tests && mkdir DIV2K

/bin/bash: line 1: cd: super-resiolution-model-tests/DIV2K: No such file or directory
/bin/bash: line 1: cd: super-resiolution-model-tests/DIV2K: No such file or directory
/bin/bash: line 1: cd: super-resiolution-model-tests/DIV2K: No such file or directory
/bin/bash: line 1: cd: super-resiolution-model-tests/DIV2K: No such file or directory


In [5]:
!cd super-resolution-model-tests//DIV2K && wget http://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_train_LR_x8.zip
!cd super-resolution-model-tests//DIV2K && wget http://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_valid_LR_x8.zip
!cd super-resolution-model-tests//DIV2K && wget http://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_train_HR.zip
!cd super-resolution-model-tests//DIV2K && wget http://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_valid_HR.zip

--2025-11-20 15:10:49--  http://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_train_LR_x8.zip
Resolving data.vision.ee.ethz.ch (data.vision.ee.ethz.ch)... 129.132.52.178, 2001:67c:10ec:36c2::178
Connecting to data.vision.ee.ethz.ch (data.vision.ee.ethz.ch)|129.132.52.178|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_train_LR_x8.zip [following]
--2025-11-20 15:10:49--  https://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_train_LR_x8.zip
Connecting to data.vision.ee.ethz.ch (data.vision.ee.ethz.ch)|129.132.52.178|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 65753171 (63M) [application/zip]
Saving to: ‘DIV2K_train_LR_x8.zip’

DIV2K_train_LR_x8.z 100%[===================>]  62.71M  19.8MB/s    in 3.7s    

2025-11-20 15:10:54 (16.7 MB/s) - ‘DIV2K_train_LR_x8.zip’ saved [65753171/65753171]

--2025-11-20 15:10:54--  http://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_valid_LR_x8.zip
Resolving data.vision

In [6]:
!cd super-resolution-model-tests//DIV2K && unzip DIV2K_train_LR_x8.zip
!cd super-resolution-model-tests//DIV2K && unzip DIV2K_valid_LR_x8.zip
!cd super-resolution-model-tests//DIV2K && unzip DIV2K_train_HR.zip
!cd super-resolution-model-tests//DIV2K && unzip DIV2K_valid_HR.zip

Archive:  DIV2K_train_LR_x8.zip
   creating: DIV2K_train_LR_x8/
  inflating: DIV2K_train_LR_x8/0678x8.png  
  inflating: DIV2K_train_LR_x8/0386x8.png  
  inflating: DIV2K_train_LR_x8/0633x8.png  
  inflating: DIV2K_train_LR_x8/0603x8.png  
  inflating: DIV2K_train_LR_x8/0110x8.png  
  inflating: DIV2K_train_LR_x8/0725x8.png  
  inflating: DIV2K_train_LR_x8/0445x8.png  
  inflating: DIV2K_train_LR_x8/0210x8.png  
  inflating: DIV2K_train_LR_x8/0599x8.png  
  inflating: DIV2K_train_LR_x8/0732x8.png  
  inflating: DIV2K_train_LR_x8/0283x8.png  
  inflating: DIV2K_train_LR_x8/0122x8.png  
  inflating: DIV2K_train_LR_x8/0021x8.png  
  inflating: DIV2K_train_LR_x8/0250x8.png  
  inflating: DIV2K_train_LR_x8/0782x8.png  
  inflating: DIV2K_train_LR_x8/0357x8.png  
  inflating: DIV2K_train_LR_x8/0586x8.png  
  inflating: DIV2K_train_LR_x8/0641x8.png  
  inflating: DIV2K_train_LR_x8/0239x8.png  
  inflating: DIV2K_train_LR_x8/0575x8.png  
  inflating: DIV2K_train_LR_x8/0480x8.png  
  inflating:

## Load in the data

In [2]:
!cd super-resolution-model-tests && git pull

remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 3 (delta 2), reused 3 (delta 2), pack-reused 0 (from 0)
Unpacking objects: 100% (3/3), 290 bytes | 290.00 KiB/s, done.
From https://github.com/ed22699/super-resolution-model-tests
   d7388e9..17fd127  main       -> origin/main
Updating d7388e9..17fd127
Fast-forward
 gan.py | 22 +---------------------
 1 file changed, 1 insertion(+), 21 deletions(-)


In [3]:
import numpy as np
from multiprocessing import cpu_count
import sys
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision
from torchvision import datasets, transforms
from torchvision.utils import save_image
import matplotlib.pyplot as plt
import time
from PIL import Image
sys.path.append("super-resolution-model-tests")
from dataloader import DIV2KDataLoader

# For GPU (CUDA) or CPU
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Load and preprocess the dataset
transformList = []
transformList.append(transforms.ToTensor())
basic_transforms = transforms.Compose(transformList)

datasetRoot = "super-resolution-model-tests/DIV2K"
lr_path = datasetRoot+"/DIV2K_train_LR_x8"
hr_path = datasetRoot+"/DIV2K_train_HR"

train_dataset = DIV2KDataLoader(
    root_dir_lr=lr_path,
    root_dir_hr=hr_path,
    transform=basic_transforms,
    mode="train",
    batch_size=50
)

lr_path = datasetRoot+"/DIV2K_valid_LR_x8"
hr_path = datasetRoot+"/DIV2K_valid_HR"

val_dataset = DIV2KDataLoader(
    root_dir_lr=lr_path,
    root_dir_hr=hr_path,
    transform=basic_transforms,
    mode="val",
    batch_size=50
)

train_loader = torch.utils.data.DataLoader(
    train_dataset,
    shuffle=True,
    batch_size=50,
    pin_memory=True,
    num_workers=cpu_count(),
)
val_loader = torch.utils.data.DataLoader(
    val_dataset,
    shuffle=False,
    batch_size=50,
    num_workers=cpu_count(),
    pin_memory=True,
)

Found 800 images for train task.
Found 800 images for train task.
Found 100 images for val task.
Found 100 images for val task.


## Train Generator and Discriminator

In [7]:

from gan import Generator, Discriminator

# Define the Binary Cross Entropy loss function
loss_func = nn.BCELoss()

def discriminator_loss(real_output, fake_output):
    # Loss for real images
    real_loss = loss_func(real_output, torch.ones_like(real_output).to(device))
    # Loss for fake images
    fake_loss = loss_func(fake_output, torch.zeros_like(fake_output).to(device))

    loss_D = real_loss + fake_loss
    return loss_D

def generator_loss(fake_output):
    # Compare discriminator's output on fake images with target labels of 1
    loss_G = loss_func(fake_output, torch.ones_like(fake_output).to(device))
    return loss_G

In [9]:

from utils.save_checkpoint import save_checkpoint

def training(x):

    '''Training step for the Discriminator'''
    real_x = x.to(device)
    real_output = gan_D(real_x)
    fake_x = gan_G(torch.randn([batch_size, input_dim]).to(device)).detach()
    fake_output = gan_D(fake_x)
    loss_D =discriminator_loss(real_output, fake_output)

    # Backpropagate the discriminator loss and update its parameters
    optim_D.zero_grad()
    loss_D.backward()
    optim_D.step()

    '''Training step for the Generator'''
    fake_x = gan_G(torch.randn([batch_size, input_dim]).to(device))
    fake_output = gan_D(fake_x)
    loss_G = generator_loss(fake_output)

    # Backpropagate the generator loss and update its parameters
    optim_G.zero_grad()
    loss_G.backward()
    optim_G.step()

    return loss_D, loss_G

ModuleNotFoundError: No module named 'utils'

In [ ]:
def visualise_generated_images(generator, input_dim, epoch, image_dir, num_images=64):
    #     """
    #     Parameters:
    #         - num_images: the number of images generated, defaults to 64.
    #     """

    # evaluation mode
    generator.eval()

    # using the generator to generate images
    noise = torch.randn(num_images, input_dim).to(device)
    with torch.no_grad():
      gen_imgs = generator(noise)

    gen_image = gen_imgs.detach().cpu()
    grid_tensor = torchvision.utils.make_grid(gen_image, normalize=True)
    grid_np = grid_tensor.permute(1, 2, 0).numpy()
    gird_scaled = (grid_np * 255).astype(np.uint8)
    filename = f"./{image_dir}/ep{epoch}.png"
    im = Image.fromarray(gird_scaled.squeeze())
    im.save(filename)
    display_image(grid_np)

def display_image(epoch_no):
    plt.imshow(epoch_no, cmap='gray')
    plt.show()
    # display a single image using the epoch number

def visualise_loss(losses_D, losses_G, image_dir, loss_type):
  plt.plot(losses_D, label="losses_D")
  plt.plot(losses_G, label="losses_G")
  plt.title(loss_type)
  plt.legend()
  plt.savefig(f"./{image_dir}/loss_type: {loss_type}.png")
  plt.show()

In [ ]:
input_dim = 100
batch_size = 128
num_epoch = 10
checkpoint_dir = './training_checkpoints'
os.makedirs(checkpoint_dir, exist_ok=True)
image_dir = './generated_image/'
os.makedirs(image_dir, exist_ok=True)

gan_G = Generator(input_dim).to(device)
gan_D = Discriminator().to(device)

# Define separate Adam optimizers for generator and discriminator
optim_G = torch.optim.Adam(gan_G.parameters(), lr=0.0002)
optim_D = torch.optim.Adam(gan_D.parameters(), lr=0.0002)

# Initialise the list to store the losses for each epoch
iteration_losses_D = []
iteration_losses_G = []
epoch_losses_D = []
epoch_losses_G = []

for epoch in range(num_epoch):
    start_time = time.time()
    total_loss_D, total_loss_G = 0, 0

    for i, (x, _) in enumerate(train_loader):
        loss_D, loss_G = training(x)

        iteration_losses_D.append(loss_D.detach().item())
        iteration_losses_G.append(loss_G.detach().item())
        total_loss_D += loss_D.detach().item()
        total_loss_G += loss_G.detach().item()

    epoch_losses_D.append(total_loss_D / len(train_loader))
    epoch_losses_G.append(total_loss_G / len(train_loader))

    # Save model checkpoints
    if (epoch + 1) % 10 == 0:
        save_checkpoint(epoch + 1, gan_G, gan_D, optim_G, optim_D, checkpoint_dir)

    # losses once per epoch
    print(f'Epoch [{epoch + 1}/{num_epoch}] | Loss_D {iteration_losses_D[-1]:.4f} | Loss_G {iteration_losses_G[-1]:.4f} | Time: {time.time() - start_time:.2f} sec')
    print(f'Epoch [{epoch + 1}/{num_epoch}]  | Loss_D {epoch_losses_D[epoch]:.4f} | Loss_G {epoch_losses_G[epoch]:.4f} | Time: {time.time() - start_time:.2f} sec')

    # Task1: visualise the generated image at different epochs
    visualise_generated_images(gan_G, input_dim, epoch, image_dir)

# Task2: visualise the loss through a plot
visualise_loss(iteration_losses_D, iteration_losses_G, image_dir, 'Iteration')
visualise_loss(epoch_losses_D, epoch_losses_G, image_dir, 'Epoch')
